In [2]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm

In [3]:
transactions = pd.read_excel("/content/drive/Shareddrives/Datathon/Datathon/InventoryAndSale_snapshot_data/Sales_snapshot_data/TT T06-2023_split_1.xlsx", dtype={'product_id':str}) # @param {type:"string"}
transactions.drop(['channel_id', 'cost_price', 'net_price'], inplace=True, axis=1)

# transactions['sold_quantity'] = 1

In [4]:
transactions.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29772 entries, 0 to 29771
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   month                      29772 non-null  int64 
 1   week                       29772 non-null  int64 
 2   site                       29772 non-null  int64 
 3   branch_id                  29772 non-null  int64 
 4   distribution_channel       29772 non-null  object
 5   distribution_channel_code  29772 non-null  object
 6   sold_quantity              29772 non-null  int64 
 7   customer_id                29772 non-null  object
 8   product_id                 29772 non-null  object
dtypes: int64(5), object(4)
memory usage: 2.0+ MB


In [5]:
transactions.head()

,month,week,site,branch_id,distribution_channel,distribution_channel_code,sold_quantity,customer_id,product_id
0,2023006,202322,1101,1100,Bán lẻ,FP,1,dd5a95cfa,1cdccfddebf04285a10c871fdf2f1ccfXNG39
1,2023006,202325,1504,1500,Bán lẻ,FP,1,41fd17be1,d468eb983cbb4849a54ed39610fb4e89XNH44
2,2023006,202323,1621,1600,Bán lẻ,FP,1,299feddd6,e29283acf2a84141b48a921cbdc46590XNH29
3,2023006,202324,1113,1100,Bán lẻ,FP,1,a0c1d36b9,c177bba7b83a4177b04954c5825cab8dXNH39
4,2023006,202322,1100,1100,Phát sinh,ZF2,1,1e094f54c,94c3f5fe7691442081f3ae9727a1eea7XAM41


In [6]:
start_week = 202322
# Filter transactions by date

transactions = transactions.loc[transactions["week"] >= start_week]

# Filter transactions by number of an article has been bought
products_bought_count = transactions[['product_id', 'week']].groupby('product_id').count().reset_index().rename(columns={'week': 'count'})
most_bought_products = products_bought_count[products_bought_count['count']>10]['product_id'].values
transactions = transactions[transactions['product_id'].isin(most_bought_products)]

In [7]:
np.random.seed(0)

negative_samples = pd.DataFrame({
    'product_id': np.random.choice(transactions.product_id.unique(), transactions.shape[0]),
    'customer_id': np.random.choice(transactions.customer_id.unique(), transactions.shape[0]),
    'sold_quantity': np.zeros(transactions.shape[0])
})

In [8]:
from sklearn.metrics.pairwise import cosine_similarity


class ItemBased_RecSys:
    ''' Collaborative filtering using a custom sim(u,u'). '''

    def __init__(self, positive_transactions, negative_transactions, num_components=10):
        ''' Constructor '''
        self.positive_transactions = positive_transactions
        self.transactions = pd.concat([positive_transactions, negative_transactions])
        self.customers = self.transactions.customer_id.values
        self.articles = self.transactions.product_id.values  # Change 'article_id' to 'product_id'
        self.sold_quantity = self.transactions.sold_quantity.values  # Change 'bought' to 'sold_quantity'
        self.num_components = num_components

        self.customer_id2index = {c: i for i, c in enumerate(np.unique(self.customers))}
        self.article_id2index = {a: i for i, a in enumerate(np.unique(self.articles))}

    def __sdg__(self):
        for idx in tqdm(self.training_indices):
            # Get the current sample
            customer_id = self.customers[idx]
            article_id = self.articles[idx]
            sold_quantity = self.sold_quantity[idx]  # Change 'bought' to 'sold_quantity'

            # Get the index of the user and the article
            customer_index = self.customer_id2index[customer_id]
            article_index = self.article_id2index[article_id]

            # Compute the prediction and the error
            prediction = self.predict_single(customer_index, article_index)
            error = (sold_quantity - prediction)  # Change 'bought' to 'sold_quantity'

            # Update latent factors in terms of the learning rate and the observed error
            self.customers_latent_matrix[customer_index] += self.learning_rate * \
                                                            (error * self.articles_latent_matrix[article_index] - \
                                                             self.lmbda * self.customers_latent_matrix[customer_index])
            self.articles_latent_matrix[article_index] += self.learning_rate * \
                                                           (error * self.customers_latent_matrix[customer_index] - \
                                                            self.lmbda * self.articles_latent_matrix[article_index])

    def fit(self, n_epochs=10, learning_rate=0.001, lmbda=0.1):
        ''' Compute the matrix factorization R = P x Q '''
        self.learning_rate = learning_rate
        self.lmbda = lmbda
        n_samples = self.transactions.shape[0]

        # Initialize latent matrices
        self.customers_latent_matrix = np.random.normal(scale=1., size=(len(np.unique(self.customers)),
                                                                       self.num_components))
        self.articles_latent_matrix = np.random.normal(scale=1., size=(len(np.unique(self.articles)),
                                                                      self.num_components))

        for epoch in range(n_epochs):
            print('Epoch: {}'.format(epoch))
            self.training_indices = np.arange(n_samples)

            # Shuffle training samples and follow stochastic gradient descent
            np.random.shuffle(self.training_indices)
            self.__sdg__()

    def predict_single(self, customer_index, article_index):
        ''' Make a prediction for a specific user and article '''
        prediction = np.dot(self.customers_latent_matrix[customer_index],
                           self.articles_latent_matrix[article_index])
        prediction = np.clip(prediction, 0, 1)

        return prediction

    def default_recommendation(self):
        ''' Calculate time decaying popularity '''
        # Calculate time decaying popularity. This leads to items bought more recently having more weight in the popularity list.
        # In simple words, item A bought 5 times on the first day of the train period is inferior than item B bought 4 times on the last day of the train period.
        self.positive_transactions['pop_factor'] = self.positive_transactions['week'].apply(lambda x: 202322 - x)
        transactions_by_article = self.positive_transactions[['product_id', 'pop_factor']].groupby(
            'product_id').sum().reset_index()  # Change 'article_id' to 'product_id'
        return transactions_by_article.sort_values(by='pop_factor', ascending=False)[
            'product_id'].values[:12]  # Change 'article_id' to 'product_id'

    def predict(self, customers):
        ''' Make recommendations '''
        recommendations = []

        # Compute similarity matrix (cosine)
        similarity_matrix = cosine_similarity(self.articles_latent_matrix, self.articles_latent_matrix,
                                              dense_output=False)

        # Convert similarity matrix into a matrix containing the 12 most similar items' index for each item
        similarity_matrix = np.argsort(similarity_matrix, axis=1)
        similarity_matrix = similarity_matrix[:, -12:]

        # Get default recommendation (time decay popularity)
        default_recommendation = self.default_recommendation()

        # Group articles by user and articles to compute the number of times each article has been bought by each user
        transactions_by_customer = self.positive_transactions[['customer_id', 'product_id', 'sold_quantity']].groupby(
            ['customer_id', 'product_id']).count().reset_index()  # Change 'article_id' to 'product_id' and 'bought' to 'sold_quantity'
        most_bought_article = transactions_by_customer.loc[
            transactions_by_customer.groupby('customer_id').sold_quantity.idxmax()][
            'product_id'].values  # Change 'article_id' to 'product_id' and 'bought' to 'sold_quantity'

        # Make predictions
        for customer in tqdm(customers):
            try:
                rec_aux1 = []
                rec_aux2 = []
                aux = []

                # Retrieve the most bought article by customer
                user_most_bought_article_id = most_bought_article[self.customer_id2index[customer]]

                # Using the similarity matrix, get the 6 most similar articles
                rec_aux1 = self.articles[similarity_matrix[self.article_id2index[user_most_bought_article_id]]]
                # Return the half of the default recommendation
                rec_aux2 = default_recommendation

                # Merge half of both recommendation lists
                for rec_idx in range(6):
                    aux.append(rec_aux2[rec_idx])
                    aux.append(rec_aux1[rec_idx])

                recommendations.append(' '.join(aux))
            except:
                # Return the default recommendation
                recommendations.append(' '.join(default_recommendation))

        return pd.DataFrame({
            'customer_id': customers,
            'prediction': recommendations,
        })


In [9]:
rec = ItemBased_RecSys(transactions, negative_samples, num_components=1000)
rec.fit(n_epochs=20)

Epoch: 0


100%|██████████| 19452/19452 [00:00<00:00, 26466.21it/s]


Epoch: 1


100%|██████████| 19452/19452 [00:00<00:00, 24546.90it/s]


Epoch: 2


...

100%|██████████| 19452/19452 [00:00<00:00, 26697.79it/s]


Epoch: 18


100%|██████████| 19452/19452 [00:00<00:00, 27280.98it/s]


Epoch: 19


100%|██████████| 19452/19452 [00:00<00:00, 26350.32it/s]


In [10]:
customers = pd.read_excel("/content/drive/Shareddrives/Datathon/Datathon/sales_and_inventory_mentor_data/MasterData/Distribution Channel.xlsx").customer_id.unique() # @param {type:"string"}

In [11]:
recommendations = rec.predict(customers)

100%|██████████| 3405/3405 [00:00<00:00, 218664.05it/s]


In [12]:
print(recommendations)

     customer_id                                         prediction
0      435043cf9  878cf8efcc2549a39a00f0951b2d066dHOG32 5785af5f...
1      db3c83bfa  878cf8efcc2549a39a00f0951b2d066dHOG32 9c202224...
2      f9bd8e994  878cf8efcc2549a39a00f0951b2d066dHOG32 72ce31c6...
3      c6fb2b695  878cf8efcc2549a39a00f0951b2d066dHOG32 5c42fd6f...
4      68be208f8  878cf8efcc2549a39a00f0951b2d066dHOG32 f1962cc0...
...          ...                                                ...
3400   411920d55  878cf8efcc2549a39a00f0951b2d066dHOG32 f1962cc0...
3401   a5e95c0dc  878cf8efcc2549a39a00f0951b2d066dHOG32 f1962cc0...
3402   f553026ed  878cf8efcc2549a39a00f0951b2d066dHOG32 f1962cc0...
3403   671b5b19f  878cf8efcc2549a39a00f0951b2d066dHOG32 f1962cc0...
3404   665390c5d  878cf8efcc2549a39a00f0951b2d066dHOG32 f1962cc0...

[3405 rows x 2 columns]
